In [1]:
# Importing necessary libraries
import os, re, json
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, PeftModel
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [2]:
# Configuration
TRAIN_CSV = "/content/mbti_train_data.csv"
MODEL_NAME = "distilbert-base-uncased"
ADAPTER_DIR = "/content/mbti_lora_distilbert"
MAX_LENGTH = 256
BATCH_SIZE = 16
LR = 2e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)


Device: cuda


In [3]:
# Preprocessing the train dataset to align with professional data
def clean_text(t):
    t = str(t)
    t = t.lower()
    t = re.sub(r"http\S+|www\S+|https\S+"," ", t)
    t = re.sub(r"[^a-zA-Z0-9\s.,!?%\-]"," ", t)
    t = re.sub(r'\s+',' ', t).strip()
    return t

keywords = [
 "team","manager","lead","product","launched","published","announced",
 "joined","hired","promotion","quarter","q1","q2","q3","q4","revenue","growth",
 "metrics","audit","compliance","customer","client","enterprise","deliver",
 "delivered","roadmap","strategy","engineer","engineering","cto","ceo","director"
]
kw_pattern = re.compile(r'\b(?:' + r'|'.join(keywords) + r')\b', re.I)

def prof_score(text, min_words=20):
    s = 0
    if kw_pattern.search(text): s += 1
    if re.search(r'\b\d{2,}\b', text): s += 1
    if re.search(r'\b(q[1-4]|quarter|quarterly)\b', text, re.I): s += 1
    if len(str(text).split()) >= min_words: s += 1
    return s

df = pd.read_csv(TRAIN_CSV)
df = df.dropna(subset=["type","posts"]).reset_index(drop=True)
df["posts"] = df["posts"].astype(str).apply(clean_text)
df["text_len"] = df["posts"].str.split().apply(len)
df["prof_score"] = df["posts"].apply(lambda x: prof_score(x, min_words=20))
df["is_professional"] = df["prof_score"] >= 2

prof_df = df[df["is_professional"]].copy().reset_index(drop=True)
print("Total loaded rows:", len(df))
print("Professional filtered rows:", len(prof_df))
print(prof_df["type"].value_counts())


Total loaded rows: 8675
Professional filtered rows: 7974
type
INFP    1690
INFJ    1350
INTP    1216
INTJ    1004
ENTP     655
ENFP     603
ISTP     305
ISFP     238
ENTJ     216
ISTJ     192
ENFJ     167
ISFJ     147
ESTP      76
ESFP      41
ESFJ      38
ESTJ      36
Name: count, dtype: int64


In [4]:
# Mapping the labels and saving the json file for later use for the test data
data_df = prof_df.copy()
le = LabelEncoder()
data_df["label"] = le.fit_transform(data_df["type"])
num_labels = len(le.classes_)
print("Num classes:", num_labels)
mapping = {"classes": list(le.classes_)}
with open("label_mapping.json","w") as f:
    json.dump(mapping, f)
print("Saved label mapping to label_mapping.json")


Num classes: 16
Saved label mapping to label_mapping.json


In [5]:
#Applying the tokenzier to the train dataset
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
ds = Dataset.from_pandas(data_df[["posts","label"]])
def tokenize_fn(batch):
    return tokenizer(batch["posts"], truncation=True, padding="max_length", max_length=MAX_LENGTH)

ds = ds.map(tokenize_fn, batched=True, remove_columns=["posts"])
ds = ds.rename_column("label","labels")
ds.set_format(type="torch", columns=["input_ids","attention_mask","labels"])
print(ds)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/7974 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 7974
})


In [6]:
#Configuring the model
base_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_lin","k_lin","v_lin"],
    bias="none",
    task_type="SEQ_CLS"
)
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()
model.to(DEVICE)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 824,080 || all params: 67,789,856 || trainable%: 1.2156


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): DistilBertSdpaAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=76

In [7]:
# Training the model
args = TrainingArguments(
    output_dir="./lora_mbti",
    per_device_train_batch_size=BATCH_SIZE,
    learning_rate=LR,
    num_train_epochs=45,
    logging_steps=40,
    save_strategy="no",
    eval_strategy="no",
    remove_unused_columns=False,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds
)

trainer.train()


Step,Training Loss
40,2.388400
80,2.242500
120,2.254600
160,2.226200
200,2.212300
240,2.293000
280,2.145400
320,1.969200
360,1.980200
400,1.971800


TrainOutput(global_step=22455, training_loss=0.6585496647005966, metrics={'train_runtime': 1863.9413, 'train_samples_per_second': 192.511, 'train_steps_per_second': 12.047, 'total_flos': 2.422677445558272e+16, 'train_loss': 0.6585496647005966, 'epoch': 45.0})

In [8]:
# Saving the model
os.makedirs(ADAPTER_DIR, exist_ok=True)
model.save_pretrained(ADAPTER_DIR)
tokenizer.save_pretrained(ADAPTER_DIR)
print("Saved adapter to:", ADAPTER_DIR)



Saved adapter to: /content/mbti_lora_distilbert


In [9]:
#Evaluating the model on train dataset
from torch.utils.data import DataLoader, TensorDataset
import torch
from sklearn.metrics import accuracy_score, f1_score, classification_report

enc_train = tokenizer(
    list(data_df["posts"].values),
    truncation=True,
    padding="max_length",
    max_length=MAX_LENGTH,
    return_tensors="pt"
)

train_dataset_eval = TensorDataset(
    enc_train["input_ids"],
    enc_train["attention_mask"],
    torch.tensor(data_df["label"].values, dtype=torch.long)
)

train_loader = DataLoader(train_dataset_eval, batch_size=BATCH_SIZE, shuffle=False)

train_preds = []
train_gts = []

model.eval()
with torch.no_grad():
    for input_ids, attention_mask, labels in train_loader:
        input_ids = input_ids.to(DEVICE)
        attention_mask = attention_mask.to(DEVICE)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        batch_preds = logits.argmax(dim=1).cpu().numpy()
        train_preds.extend(batch_preds)
        train_gts.extend(labels.numpy())

#Evalution metrics
train_acc = accuracy_score(train_gts, train_preds)
train_f1 = f1_score(train_gts, train_preds, average="macro")

print(f"\nTraining Accuracy: {train_acc:.4f}")
print(f"Training F1-Macro: {train_f1:.4f}")

print("\nClassification Report:\n")
print(classification_report(train_gts, train_preds))



Training Accuracy: 0.9989
Training F1-Macro: 0.9992

Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       167
           1       1.00      1.00      1.00       603
           2       1.00      1.00      1.00       216
           3       1.00      1.00      1.00       655
           4       1.00      1.00      1.00        38
           5       1.00      1.00      1.00        41
           6       1.00      1.00      1.00        36
           7       1.00      1.00      1.00        76
           8       1.00      1.00      1.00      1350
           9       1.00      1.00      1.00      1690
          10       1.00      1.00      1.00      1004
          11       1.00      1.00      1.00      1216
          12       1.00      1.00      1.00       147
          13       1.00      1.00      1.00       238
          14       1.00      0.99      1.00       192
          15       1.00      1.00      1.00       305

  

In [10]:
# Load base model then wrap with PeftModel
base = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)
model = PeftModel.from_pretrained(base, ADAPTER_DIR, local_files_only=True)
model.to(DEVICE)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(ADAPTER_DIR, local_files_only=True)
with open("label_mapping.json","r") as f:
    label_map = json.load(f)["classes"]
label2id = {c:i for i,c in enumerate(label_map)}
id2label = {i:c for c,i in label2id.items()}


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
#Preparing test dataset for mbti prediction
test = pd.read_csv('/content/mbti_test_data.csv').rename(columns={"recent_activity":"posts","inferred_mbti":"type"})
test = test.dropna(subset=["posts","type"]).reset_index(drop=True)
test["posts"] = test["posts"].astype(str).apply(clean_text)
test["label"] = test["type"].map(label2id)
test = test.dropna(subset=["label"]).reset_index(drop=True)
test["label"] = test["label"].astype(int)
print(test.head(5))
print("Test rows after mapping to trained classes:", len(test))
print(test["type"].value_counts())


                                               posts  type  label
0  q3 engineering metrics are in. we ve hit 99.99...  INTJ     10
1  just published a new blog post on the why behi...  INFP      9
2  completed the quarterly compliance audit for o...  INTJ     10
3  hot take is the traditional banking model comp...  ENTP      3
4  so incredibly proud to announce our team s gre...  ENFP      1
Test rows after mapping to trained classes: 51
type
INTJ    18
INFJ     5
ENTP     4
INTP     4
ENFP     4
ESTP     3
INFP     2
ESFJ     2
ENTJ     2
ISTP     2
ENFJ     1
ISFP     1
ISFJ     1
ESFP     1
ISTJ     1
Name: count, dtype: int64


In [12]:
# Output prediction on test data
from torch.utils.data import DataLoader, TensorDataset

enc = tokenizer(list(test["posts"].values), truncation=True, padding="max_length", max_length=MAX_LENGTH, return_tensors="pt")
dataset = TensorDataset(enc["input_ids"], enc["attention_mask"], torch.tensor(test["label"].values, dtype=torch.long))
loader = DataLoader(dataset, batch_size=BATCH_SIZE)

preds = []
gts = []

with torch.no_grad():
    for input_ids, attention_mask, labels in loader:
        input_ids = input_ids.to(DEVICE)
        attention_mask = attention_mask.to(DEVICE)
        out = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = out.logits
        batch_preds = logits.argmax(dim=1).cpu().numpy()
        preds.extend(batch_preds)
        gts.extend(labels.numpy())

# Test data evaluation
acc = accuracy_score(gts, preds)
f1 = f1_score(gts, preds, average="macro")
print("Accuracy:", acc)
print("F1-macro:", f1)
print("\nClassification report:\n")
print(classification_report(gts, preds, target_names=[id2label[i] for i in range(len(id2label))]))


Accuracy: 0.49019607843137253
F1-macro: 0.2655455655455655

Classification report:



ValueError: Number of classes, 15, does not match size of target_names, 16. Try specifying the labels parameter

In [ ]:
import shutil
from google.colab import files

FOLDER_PATH = "/content/mbti_lora_distilbert"
ZIP_NAME = "mbti_lora_distilbert"

shutil.make_archive(ZIP_NAME, 'zip', FOLDER_PATH)

files.download(ZIP_NAME + ".zip")

In [ ]:
# import zipfile

# zip_path = "mbti_lora_distilbert.zip"
# extract_dir = "/content/mbti_lora_distilbert"

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_dir)

# print("Extracted to:", extract_dir)